In [3]:
import math
import matplotlib.pyplot as plt
import keras
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers import *
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from keras.callbacks import EarlyStopping
df=pd.read_csv("/content/BANKASIA.csv",parse_dates=['DATE'])
df.head()

#       DATE TRADING CODE  LTP*  ...   YCP  TRADE  VALUE (mn)   VOLUME
0  439 2018-05-08     BANKASIA  17.1  ...  17.5    158       6.782  400,155
1  438 2018-06-08     BANKASIA  16.9  ...  16.8    160      11.494  673,813
2  437 2018-07-08     BANKASIA  17.2  ...  16.9    218      16.415  955,798
3  436 2018-08-08     BANKASIA  17.4  ...  17.2    242      16.895  982,419
4  435 2018-09-08     BANKASIA  17.3  ...  17.3    336      14.668  839,199

[5 rows x 12 columns]

In [4]:
# Sort DataFrame by date
df = df.sort_values('DATE')

# Double check the result
df.head()

#       DATE TRADING CODE  LTP*  ...   YCP  TRADE  VALUE (mn)   VOLUME
36  403 2018-01-10     BANKASIA  16.7  ...  16.8     57       2.271  135,168
59  380 2018-01-11     BANKASIA  17.1  ...  17.4    134       4.479  257,851
37  402 2018-02-10     BANKASIA  16.7  ...  16.8     52       1.806  107,644
79  360 2018-02-12     BANKASIA  17.5  ...  17.3     37       1.489   85,673
16  423 2018-03-09     BANKASIA  17.4  ...  17.5    262      11.665  675,112

[5 rows x 12 columns]

In [5]:
df = df.set_index('DATE')
df.head(3)

# TRADING CODE  LTP*  HIGH  ...   YCP  TRADE  VALUE (mn)   VOLUME
DATE                                      ...                                  
2018-01-10  403     BANKASIA  16.7  17.0  ...  16.8     57       2.271  135,168
2018-01-11  380     BANKASIA  17.1  17.8  ...  17.4    134       4.479  257,851
2018-02-10  402     BANKASIA  16.7  16.9  ...  16.8     52       1.806  107,644

[3 rows x 11 columns]

In [6]:
df['daily_ret'] = df['CLOSEP*'].pct_change()
df['daily_ret'] = df['daily_ret'].fillna(0.0)

In [7]:
df['volatility']= df['daily_ret'].rolling(window=21).std()
df['volatility'] = df['volatility'].fillna(0.0)

In [8]:
df['volatility'] = df['volatility']*100

In [9]:
import matplotlib.pyplot as plt 

In [10]:
df=df.reset_index()['volatility']

In [11]:
df.shape

(439,)

In [12]:
from sklearn.preprocessing import MinMaxScaler
scaler=MinMaxScaler(feature_range=(0,1))
df=scaler.fit_transform(np.array(df).reshape(-1,1))

In [13]:
##splitting dataset into train and test split where train=70 percent and test=30 percent
training_size=int(len(df)*0.70)
test_size=len(df)-training_size
train_data,test_data=df[0:training_size,:],df[training_size:len(df),:1]




In [14]:
import numpy
# convert an array of values into a dataset matrix
def create_dataset(dataset, time_step=1):
	dataX, dataY = [],[]
	for i in range(len(dataset)-time_step-1):
		a = dataset[i:(i+time_step), 0]   ###i=0, 0,1,2,3-----99   100 
		dataX.append(a)
		dataY.append(dataset[i + time_step, 0])
	return numpy.array(dataX), numpy.array(dataY)

In [15]:

# reshape into X=t,t+1,t+2,t+3 and Y=t+4
time_step = 100
X_train, y_train = create_dataset(train_data, time_step)
X_test, ytest = create_dataset(test_data, time_step)

In [16]:
from sklearn.ensemble import RandomForestRegressor,AdaBoostRegressor,GradientBoostingRegressor
import xgboost as xgb
from sklearn.datasets import make_regression

In [17]:
model1 = RandomForestRegressor(n_estimators=1000)
model2 =AdaBoostRegressor(n_estimators=1000,learning_rate=0.01)
model3= GradientBoostingRegressor(n_estimators=1000,learning_rate=0.01)
model4 = xgb.XGBRegressor(n_estimators=1000,learning_rate=0.01)



In [18]:
model1.fit(X_train, y_train)

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=1000, n_jobs=None, oob_score=False,
                      random_state=None, verbose=0, warm_start=False)

In [19]:
model2.fit(X_train, y_train)

AdaBoostRegressor(base_estimator=None, learning_rate=0.01, loss='linear',
                  n_estimators=1000, random_state=None)

In [20]:
model3.fit(X_train, y_train)

GradientBoostingRegressor(alpha=0.9, ccp_alpha=0.0, criterion='friedman_mse',
                          init=None, learning_rate=0.01, loss='ls', max_depth=3,
                          max_features=None, max_leaf_nodes=None,
                          min_impurity_decrease=0.0, min_impurity_split=None,
                          min_samples_leaf=1, min_samples_split=2,
                          min_weight_fraction_leaf=0.0, n_estimators=1000,
                          n_iter_no_change=None, presort='deprecated',
                          random_state=None, subsample=1.0, tol=0.0001,
                          validation_fraction=0.1, verbose=0, warm_start=False)

In [21]:
model4.fit(X_train, y_train)

[13:37:13] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0,
             importance_type='gain', learning_rate=0.01, max_delta_step=0,
             max_depth=3, min_child_weight=1, missing=None, n_estimators=1000,
             n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
             silent=None, subsample=1, verbosity=1)

In [22]:

train_predict1=model1.predict(X_train)
test_predict1= model1.predict(X_test)

In [23]:

train_predict2=model2.predict(X_train)
test_predict2= model2.predict(X_test)

In [24]:

train_predict3=model3.predict(X_train)
test_predict3= model3.predict(X_test)

In [25]:

train_predict4=model4.predict(X_train)
test_predict4= model4.predict(X_test)

In [26]:
ytest

array([0.60056469, 0.51743445, 0.51479791, 0.39498161, 0.39558728,
       0.39558728, 0.39558728, 0.39558728, 0.39558728, 0.39748225,
       0.39961565, 0.39961565, 0.40038649, 0.47250859, 0.53418081,
       0.53217332, 0.53429588, 0.48227324, 0.43208929, 0.55618627,
       0.54552943, 0.55436001, 0.61378156, 0.6950148 , 0.69303818,
       0.81700536, 0.88128003, 0.886649  , 0.90033111, 0.93473037,
       0.99813441])

In [27]:
Y_test = np.reshape(ytest,(1, ytest.size))

In [28]:
Y_test =scaler.inverse_transform(Y_test)

In [29]:
Y_test

array([[3.53212995, 3.04321207, 3.02770565, 2.3230243 , 2.32658648,
        2.32658648, 2.32658648, 2.32658648, 2.32658648, 2.33773148,
        2.35027871, 2.35027871, 2.35481227, 2.77898744, 3.14170329,
        3.12989653, 3.14238003, 2.8364168 , 2.54126749, 3.27112501,
        3.20844844, 3.26038413, 3.60986297, 4.08762393, 4.07599872,
        4.80509285, 5.18311458, 5.21469138, 5.29516064, 5.49747462,
        5.87037587]])

In [30]:
test_predict1

array([0.65776247, 0.66787896, 0.55714716, 0.53848197, 0.38542137,
       0.38509607, 0.38342486, 0.38151392, 0.37935135, 0.37952419,
       0.38228746, 0.38484996, 0.38622443, 0.38998228, 0.50063898,
       0.55396632, 0.55587542, 0.55742564, 0.50919052, 0.41732253,
       0.55922744, 0.56232327, 0.56152536, 0.68561835, 0.69201747,
       0.71135801, 0.79663414, 0.84586814, 0.84695476, 0.84625928,
       0.84547164])

In [31]:
test_predict1 = np.reshape(test_predict1,(1, test_predict1.size))
test_predict2 = np.reshape(test_predict2,(1, test_predict2.size))
test_predict3 = np.reshape(test_predict3,(1, test_predict3.size))
test_predict4 = np.reshape(test_predict4,(1, test_predict3.size))



In [32]:
test_predict1 =scaler.inverse_transform(test_predict1)
test_predict2 =scaler.inverse_transform(test_predict2)
test_predict3 =scaler.inverse_transform(test_predict3)
test_predict4 =scaler.inverse_transform(test_predict4)

In [33]:
test_predict1

array([[3.86853002, 3.92802861, 3.27677636, 3.16699985, 2.26679722,
        2.264884  , 2.25505507, 2.24381615, 2.23109735, 2.23211383,
        2.24836558, 2.26343658, 2.2715203 , 2.29362155, 2.94443211,
        3.25806874, 3.26929679, 3.27841421, 2.99472664, 2.45441902,
        3.28901123, 3.30721885, 3.30252605, 4.03236017, 4.06999557,
        4.18374403, 4.68528263, 4.97484491, 4.98123574, 4.97714539,
        4.97251295]])

In [34]:
test_predict2

array([[3.90942045, 3.91823669, 3.22814005, 3.2074762 , 2.31786645,
        2.31442367, 2.30637774, 2.25577022, 2.25573866, 2.25524119,
        2.27405363, 2.29147008, 2.29414706, 2.30637774, 3.10114703,
        3.18198077, 3.18704074, 3.18817462, 3.10912628, 2.43344263,
        3.19717869, 3.20038417, 3.20634168, 3.9967799 , 3.999188  ,
        4.0267873 , 4.78263236, 4.93346117, 4.93346117, 4.93346117,
        4.93346117]])

In [35]:
test_predict3

array([[3.69151847, 3.71931601, 3.26644232, 3.04771437, 2.40011468,
        2.39960524, 2.37282716, 2.31044702, 2.33907162, 2.29089869,
        2.34230184, 2.34851334, 2.3373453 , 2.34973271, 2.94396044,
        3.20251375, 3.28483816, 3.27953153, 3.03615071, 2.62651365,
        3.36693081, 3.33591044, 3.33438673, 3.83414262, 3.90269345,
        4.10921866, 4.64479413, 4.83009519, 4.83842891, 4.82298398,
        4.81306247]])

In [36]:
test_predict4

array([[3.7072968, 3.7245064, 3.3166647, 3.055003 , 2.4433668, 2.444178 ,
        2.419301 , 2.3937314, 2.3470218, 2.2933176, 2.360002 , 2.3876624,
        2.3573458, 2.3598094, 2.9437344, 3.2568605, 3.3134332, 3.2853718,
        2.9913447, 2.5533462, 3.3809807, 3.341132 , 3.3440943, 3.7533827,
        3.9360976, 4.034068 , 4.566617 , 4.769195 , 4.785321 , 4.7997627,
        4.7968593]], dtype=float32)

In [46]:
Y_test

array([[3.53212995, 3.04321207, 3.02770565, 2.3230243 , 2.32658648,
        2.32658648, 2.32658648, 2.32658648, 2.32658648, 2.33773148,
        2.35027871, 2.35027871, 2.35481227, 2.77898744, 3.14170329,
        3.12989653, 3.14238003, 2.8364168 , 2.54126749, 3.27112501,
        3.20844844, 3.26038413, 3.60986297, 4.08762393, 4.07599872,
        4.80509285, 5.18311458, 5.21469138, 5.29516064, 5.49747462,
        5.87037587]])

In [37]:
#Y_test = Y_test.transpose()
RMSE1 = math.sqrt(mean_squared_error(Y_test,test_predict1))
RMSE1

0.4049304417192163

In [38]:
#Y_test = Y_test.transpose()
RMSE2 = math.sqrt(mean_squared_error(Y_test,test_predict2))
RMSE2

0.42087383903499753

In [39]:
#Y_test = Y_test.transpose()
RMSE3 = math.sqrt(mean_squared_error(Y_test,test_predict3))
RMSE3

0.40523414103235655

In [40]:
#Y_test = Y_test.transpose()
RMSE4 = math.sqrt(mean_squared_error(Y_test,test_predict4))
RMSE4

0.4268645225573781

In [41]:

from sklearn.metrics import mean_absolute_error

mae = mean_absolute_error(Y_test,test_predict1)
print('MAE: %f' % mae)

MAE: 0.300008


In [42]:
from sklearn.metrics import mean_absolute_error

mae = mean_absolute_error(Y_test,test_predict2)
print('MAE: %f' % mae)

MAE: 0.294531


In [43]:
from sklearn.metrics import mean_absolute_error

mae = mean_absolute_error(Y_test,test_predict3)
print('MAE: %f' % mae)

MAE: 0.298795


In [44]:
from sklearn.metrics import mean_absolute_error

mae = mean_absolute_error(Y_test,test_predict4)
print('MAE: %f' % mae)

MAE: 0.323399
